In [16]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from pymongo import MongoClient
from bson.json_util import dumps
#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDAAC import AnimalShelter

###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32272, 'AAC', 'animals')

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
#print(df)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Robert Lulashi')


image_filename = 'GSLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Robert Lulashi CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px', 'height': 'auto'})
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    html.Hr(),
    html.Div(
 className='row',
        style={'display': 'flex'},
        children=[
                html.Button(id='button-one',n_clicks=0, children= 'Water Rescue'),
                html.Button(id='button-two',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
                html.Button(id='button-three',n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='button-four', n_clicks=0, children='reset')          
        ]

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                 for i in df.columns],
        
        data=df.to_dict('records'),
        editable =False,
        filter_action ='native',
        sort_action ='native',
        sort_mode ='multi',
        column_selectable =False,
        row_selectable ='single',
        row_deletable =False,
        selected_columns =[],
        selected_rows =[],
        page_action ='native',
        page_current =0,
        page_size =10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data')],
              [Input('button-one', 'n_clicks'),Input('button-two','n_clicks'), 
               Input('button-three','n_clicks'),Input('button-four','n_clicks')])
def update_dashboard(button1,button2,button3,button4):
### FIX ME Add code to filter interactive data table with MongoDB queries
    df = pd.DataFrame.from_records(shelter.read({}))
    
    if (int(button1) >= 1):
        df = pd.DataFrame(list(shelter.read({'$and': 
                                            [{'sex_upon_outcome': 'Intact Female'},
                                                      {'$or': [
                                                          {'breed': 'Labrador Retriever Mix'},
                                                          {'breed': 'Chesa Bay Retr Mix'},
                                                          {'breed': 'Newfoundland'}],                                                                                                                },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                             
                                               })))
                                                         
    elif (int(button2)>= 1):
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                      {'$or': [
                                                          {'breed': 'German Shepherd'},
                                                          {'breed': 'Alaskan Malamute'},
                                                          {'breed': 'Old English Sheepdog'},
                                                          {'breed': 'Rottweiler'},
                                                          {'breed': 'Siberian Husky'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                })))

    elif (int(button3)>=1):
        df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                      {'$or': [
                                                          {'breed': 'Doberman Pinscher'},
                                                          {'breed': 'German Shepherd'},
                                                          {'breed': 'Golden Retriever'},
                                                          {'breed': 'Bloodhound'},
                                                          {'breed': 'Rottweiler'}]
                                                      },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                })))
    elif(int(button4)>=1):
        df = pd.DataFrame.from_records(shelter.read())
        

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    df.drop(columns=['_id'],inplace=True)
    return (data)



# Display the breeds of animal based on quantity represented in
# the data table

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])


def update_graphs(viewData):
    
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None:
        return html.Div("Loading Application... Please wait...")
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if len(index) == 0:
        return html.Div("No row selected")
    
    row = index[0]
    
    if row >= len(dff):
        return html.Div("Selected row index is out of range")
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connection Successful
Dash app running on http://127.0.0.1:31271/
